# Lab 2: Códigos Cíclicos

Alunos: Fernando Zanchitta e Guilherme Kowalczuk

# 1. Simulação

In [1]:
from channel import BSC
from encoder import NaiveEncoder, ConvolutionalEncoder
from lab1 import Simulator
from helpers import safe_load

import numpy as np
import random
random.seed(2023)

p = [5e-1, 2e-1, 1e-1, 5e-2, 2e-2, 1e-2, 5e-3, 2e-3, 1e-3, 5e-4, 2e-4, 1e-4, 5e-5, 2e-5, 1e-5]

## 1.1. Naive Encoder

In [2]:
k = 4
l = 1000000//k
naive_encoder = NaiveEncoder()

channels = map(BSC, p)
sim = map(lambda channel: Simulator(naive_encoder, channel, k=k, l=l), channels)
simulated = zip(p, map(Simulator.simulate, sim))
naive_sim = safe_load('lab3_values/naive.pickle', lambda: list(simulated))

print(naive_sim)

[(0.5, 0.500831), (0.2, 0.199307), (0.1, 0.100258), (0.05, 0.050281), (0.02, 0.020018), (0.01, 0.009992), (0.005, 0.004897), (0.002, 0.001971), (0.001, 0.001009), (0.0005, 0.000483), (0.0002, 0.000207), (0.0001, 0.000105), (5e-05, 4.1e-05), (2e-05, 2e-05), (1e-05, 1.2e-05)]


## 1.2. Convolutional Encoder

In [3]:
def simulate_convolutional(m,G,k=None):
    n = 3
    conv_encoder = ConvolutionalEncoder(n, m, G)

    conv_sim = []
    for prob in p:
        if k is None:
            sim_k = int(8000/prob) # min(int(8000/prob), 1_000_000)
        else:
            sim_k = k
        channel = BSC(prob)
        sim = Simulator(conv_encoder, channel, k=sim_k, l=1)
        simulated = sim.simulate()
        conv_sim.append(simulated)

    return list(zip(p, conv_sim))

mG = [
    (3, np.array([[1,0,1,1], [1,1,0,1], [1,1,1,1]])),
    (4, np.array([[1,0,1,0,1], [1,1,0,1,1], [1,1,1,1,1]])), 
    (6, np.array([[1,0,0,1,1,1,1], [1,0,1,0,1,1,1], [1,1,0,1,1,0,1]])),
]

conv_sim = []
for m,G in mG:
    sim = safe_load(f'lab3_values/conv{m}.pickle', lambda: simulate_convolutional(m,G))
    conv_sim.append(sim)
    print(f'm={m}')
    print(sim)

m=3
[(0.5, 0.505), (0.2, 0.206), (0.1, 0.003), (0.05, 0.0), (0.02, 0.0), (0.01, 0.0), (0.005, 0.0), (0.002, 0.0), (0.001, 0.0), (0.0005, 0.0), (0.0002, 0.0), (0.0001, 0.0), (5e-05, 0.0), (2e-05, 0.0), (1e-05, 0.0)]


 54%|█████▍    | 8650/16000 [00:03<00:02, 2714.29it/s]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = plt.gca()
# plt.plot(p, naive_sim)
plt.plot([k for k,_ in conv_sim[0]], [v for _,v in conv_sim[0]])
ax.set_yscale('log')
ax.set_xscale('log')
ax.invert_xaxis()